In [1]:
import pandas as pd

In [8]:
accounts = pd.read_csv('account_info.csv')
transactions = pd.read_csv('account_transaction.csv')

In [9]:
accounts.head()

,account_no,pos_dt,opn_dt,cls_dt,drmt_dys
0,3450015338,2016-10-31 00:00:00.0,2007-01-29 00:00:00.0,NaN,1
1,3450046050,2016-10-31 00:00:00.0,2012-03-30 00:00:00.0,NaN,4
2,3450021593,2016-10-31 00:00:00.0,2005-02-24 00:00:00.0,NaN,0
3,3450004723,2016-10-31 00:00:00.0,2016-05-29 00:00:00.0,NaN,0
4,3450001685,2016-10-31 00:00:00.0,2016-01-06 00:00:00.0,NaN,0


In [13]:
accounts.describe()

,account_no,drmt_dys
count,5.400000e+03,5400.000000
mean,3.450022e+09,45.688889
std,1.527381e+04,312.581628
min,3.450000e+09,-1.000000
25%,3.450009e+09,0.000000
50%,3.450019e+09,1.000000
75%,3.450037e+09,12.000000
max,3.450050e+09,4835.000000


In [14]:
accounts.describe(include=['O'])

,pos_dt,opn_dt,cls_dt
count,5400,5400,95
unique,3,2820,42
top,2016-10-31 00:00:00.0,1984-10-31 00:00:00.0,2016-09-06 00:00:00.0
freq,5320,24,7


In [10]:
transactions.head()

,account_no,fm_to_account_no,txn_amt,txn_dt,txn_tm,txn_type
0,3450040137,3450082902,5000,2016-03-02 00:00:00.0,NaN,DR
1,3450011108,0,100,2016-05-28 00:00:00.0,NaN,DR
2,3450007981,3451020228,3800,2016-06-09 00:00:00.0,NaN,DR
3,3450032069,0,10000,2016-04-06 00:00:00.0,NaN,DR
4,3450003873,0,100,2016-08-05 00:00:00.0,NaN,DR


In [15]:
transactions.describe()

,account_no,fm_to_account_no,txn_amt,txn_tm
count,7.970890e+05,7.970890e+05,7.970890e+05,0.0
mean,3.450022e+09,1.639429e+09,2.013894e+04,NaN
std,1.432403e+04,1.723151e+09,1.820097e+05,NaN
min,3.450000e+09,0.000000e+00,1.000000e+02,NaN
25%,3.450011e+09,0.000000e+00,4.000000e+02,NaN
50%,3.450021e+09,0.000000e+00,2.000000e+03,NaN
75%,3.450034e+09,3.450487e+09,1.000000e+04,NaN
max,3.450050e+09,3.451196e+09,1.250000e+08,NaN


In [16]:
transactions.describe(include=['O'])

,txn_dt,txn_type
count,797089,797089
unique,261,2
top,2016-09-30 00:00:00.0,DR
freq,5074,529515


In [17]:
transactions['txn_type'].value_counts()

DR    529515
CR    267574
Name: txn_type, dtype: int64

In [18]:
transactions_dr = transactions[transactions['txn_type'] == 'DR']
transactions_cr = transactions[transactions['txn_type'] == 'CR']

In [169]:
acc_cr = transactions_cr[['account_no', 'txn_amt']].groupby('account_no').agg(['mean', 'count', 'sum'])

acc_cr2 = pd.DataFrame({
    'account_no': acc_cr.index.values,
    'cr_mean': acc_cr['txn_amt']['mean'],
    'cr_count': acc_cr['txn_amt']['count'],
    'cr_sum': acc_cr['txn_amt']['sum']
})

In [170]:
acc_dr = transactions_dr[['account_no', 'txn_amt']].groupby('account_no').agg(['mean', 'count', 'sum'])

acc_dr2 = pd.DataFrame({
    'account_no': acc_dr.index.values,
    'dr_mean': acc_dr['txn_amt']['mean'],
    'dr_count': acc_dr['txn_amt']['count'],
    'dr_sum': acc_dr['txn_amt']['sum']
})
# acc_dr['account_no'] = acc_dr.index.values
# acc_dr['dr_mean'] = acc_dr['txn_amt']['mean']
# acc_dr['dr_count'] = acc_dr['txn_amt']['count']
# acc_dr['dr_sum'] = acc_dr['txn_amt']['sum']

# acc_dr.drop('txn_amt',inplace=True,axis=1)

In [171]:
# Merge 2 Tables
acc_crdr = acc_cr2.merge(acc_dr2, how='outer', on='account_no')
acc_crdr = acc_crdr.fillna(0)

In [172]:
acc_crdr.head()
acc_crdr['crdr_sum'] = acc_crdr['cr_sum'] - acc_crdr['dr_sum']
acc_crdr['crdr_mean'] = acc_crdr['cr_mean'] - acc_crdr['dr_mean']
acc_crdr['crdr_count'] = acc_crdr['cr_count'] - acc_crdr['dr_count']

In [173]:
testdata = pd.read_csv('tj_04_test.csv', header=None)
traindata = pd.read_csv('tj_04_training.csv', header=None)

In [174]:
traindata.columns = ['account_no', 'label']
testdata.columns = ['account_no']

In [175]:
traindata_full = traindata.merge(acc_crdr, how='left', on='account_no')
testdata_full = testdata.merge(acc_crdr, how='left', on='account_no')

In [176]:
traindata_full.head(10)

,account_no,label,cr_count,cr_mean,cr_sum,dr_count,dr_mean,dr_sum,crdr_sum,crdr_mean,crdr_count
0,3450002243,sa,124.0,18208.870968,2257900.0,13.0,116700.000000,1517100.0,740800.0,-98491.129032,111.0
1,3450013179,sa,42.0,20876.190476,876800.0,187.0,5729.946524,1071500.0,-194700.0,15146.243952,-145.0
2,3450005173,sa,97.0,20590.721649,1997300.0,72.0,21976.388889,1582300.0,415000.0,-1385.667239,25.0
3,3450034206,sa,36.0,16150.000000,581400.0,104.0,6490.384615,675000.0,-93600.0,9659.615385,-68.0
4,3450033428,sa,38.0,13894.736842,528000.0,139.0,9005.755396,1251800.0,-723800.0,4888.981446,-101.0
5,3450013510,ca,24.0,28191.666667,676600.0,14.0,16342.857143,228800.0,447800.0,11848.809524,10.0
6,3450008002,sa,52.0,12473.076923,648600.0,166.0,2729.518072,453100.0,195500.0,9743.558851,-114.0
7,3450047667,sa,104.0,374771.153846,38976200.0,191.0,238513.612565,45556100.0,-6579900.0,136257.541281,-87.0
8,3450014208,sa,94.0,7643.617021,718500.0,95.0,6356.842105,603900.0,114600.0,1286.774916,-1.0
9,3450048058,ca,10.0,36230.000000,362300.0,8.0,17412.500000,139300.0,223000.0,18817.500000,2.0


In [177]:
# Let sa = 0, and ca = 1
traindata_full['label'] = traindata_full['label'].replace({'sa': 0, 'ca': 1})

In [178]:
traindata_full.head(10)

,account_no,label,cr_count,cr_mean,cr_sum,dr_count,dr_mean,dr_sum,crdr_sum,crdr_mean,crdr_count
0,3450002243,0,124.0,18208.870968,2257900.0,13.0,116700.000000,1517100.0,740800.0,-98491.129032,111.0
1,3450013179,0,42.0,20876.190476,876800.0,187.0,5729.946524,1071500.0,-194700.0,15146.243952,-145.0
2,3450005173,0,97.0,20590.721649,1997300.0,72.0,21976.388889,1582300.0,415000.0,-1385.667239,25.0
3,3450034206,0,36.0,16150.000000,581400.0,104.0,6490.384615,675000.0,-93600.0,9659.615385,-68.0
4,3450033428,0,38.0,13894.736842,528000.0,139.0,9005.755396,1251800.0,-723800.0,4888.981446,-101.0
5,3450013510,1,24.0,28191.666667,676600.0,14.0,16342.857143,228800.0,447800.0,11848.809524,10.0
6,3450008002,0,52.0,12473.076923,648600.0,166.0,2729.518072,453100.0,195500.0,9743.558851,-114.0
7,3450047667,0,104.0,374771.153846,38976200.0,191.0,238513.612565,45556100.0,-6579900.0,136257.541281,-87.0
8,3450014208,0,94.0,7643.617021,718500.0,95.0,6356.842105,603900.0,114600.0,1286.774916,-1.0
9,3450048058,1,10.0,36230.000000,362300.0,8.0,17412.500000,139300.0,223000.0,18817.500000,2.0


In [179]:
# Predict
from sklearn.model_selection import cross_val_score, train_test_split

In [235]:
# Settings
use_columns = [
#                 'cr_count',
#                 'cr_sum',
#                 'cr_mean',
                'dr_count',
#                 'dr_sum',
                'dr_mean',
#                 'crdr_sum',
#                 'crdr_mean',
                'crdr_count'
]

target_column = 'label'

# from sklearn.naive_bayes import GaussianNB
# model = GaussianNB()

# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=100)

from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, learning_rate=0.2)

# from sklearn import svm
# model = svm.SVC()

# model = XGBClassifier(max_depth=5)

testsze = 0.3

In [236]:
# Test Model
x_train, x_test, y_train, y_test = train_test_split(
    traindata_full[use_columns],
    traindata_full[target_column], test_size = testsze)

estimator = model.fit(x_train, y_train)

cross_val_score(estimator, x_test, y_test, cv=10)

array([ 0.90839695,  0.95419847,  0.91538462,  0.86923077,  0.89147287,
        0.89922481,  0.86821705,  0.91472868,  0.89147287,  0.89147287])

In [232]:
# from sklearn.model_selection import GridSearchCV
# parameters = {'learning_rate':[0.01,0.05,0.1,0.2,0.3,0.5,0.7,1], 'n_estimators':[10,30,50,70,100,200,500,1000]}
# model = AdaBoostClassifier()
# clf = GridSearchCV(model, parameters)
# clf.fit(x_train, y_train)
# print(clf.best_params_)
# print(clf.cv_results_)

In [241]:
# from sklearn.feature_selection import SelectFromModel
# model = SelectFromModel(estimator, prefit=True)

# estimator.feature_importances_

SelectFromModel(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.2, n_estimators=100, random_state=None),
        prefit=True, threshold=None)

In [238]:
# Save File
predict_y = estimator.predict(testdata_full[use_columns])

# Convert back
predict_df = pd.DataFrame(predict_y)
predict_df = predict_df.replace({0: 'sa', 1: 'ca'})

import numpy as np
np.savetxt('4.txt', predict_df[0].values, fmt="%s")